In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import copy
from keras.models import load_model
from keras.models import Sequential

In [ ]:
# -------------------------------------------------
# Constant
SIZE_FILTER_X = 36
SIZE_FILTER_Y = 36
NB_ITERATION = 10

In [ ]:
def load_model_keras(path_to_model):
    return load_model(path_to_model)

In [ ]:
def search_visage(gray_image, size_filter_X, size_filter_Y, model):
    
    # Raise an exception, if we can't apply the fitlter
    width, height = gray_image.shape[0], gray_image.shape[1]
    
    if width < size_filter_Y and height < size_filter_X :
        raise Exception ("impossible to crop properly")
        
    if (size_filter_X/2) % 2 != 0 or (size_filter_Y/2) % 2 != 0:
        raise Exception ("All dimension of the filter should be pair")
    
    # loop on the image.
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    result = []
    listImgPos = []
    for y in range(fil_divi_2_X, height - fil_divi_2_X, int(fil_divi_2_X/3)):
        for x in range(fil_divi_2_Y, width - fil_divi_2_Y, int(fil_divi_2_Y/3)):
            crop_img = gray_image[y - fil_divi_2_Y: y + fil_divi_2_Y, x - fil_divi_2_X : x + fil_divi_2_X]
            
            crop_imag_copy  = copy.deepcopy(crop_img)
            
            crop_img = np.array(crop_img)
            crop_img.resize((1,36,36,1))
            
            if int(model.predict(crop_img,verbose = 0)[0] + 0.0005) == 1:
                print ("GET A VISAGE")
                result.append((x,y))
                listImgPos.append(crop_imag_copy)
            
    return result, listImgPos

In [ ]:
def draw_rectangle(img, cor_x, cor_y, size_filter_X, size_filter_Y ):
    print("ADD RECTANGLE")
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    cv2.rectangle(img, (cor_x - fil_divi_2_X, cor_y - fil_divi_2_Y), (cor_x + fil_divi_2_X, cor_y + fil_divi_2_Y), (255,0,0), 2)
    return img

In [ ]:
def pyramid_search (path_image, size_filter_X, size_filter_Y, nb_iterations, model):
    # We get and tranform the image in gray.
    img_search = cv2.imread(path_image)
    gray_image = cv2.cvtColor(img_search, cv2.COLOR_BGR2GRAY)
    gray_imag_rect  = copy.deepcopy(gray_image)
    
    # Make the reduction.
    # --> search
    height, width = gray_image.shape
    nbPixelToReduceX = int((width  - size_filter_X) / nb_iterations)
    nbPixelToReduceY = int((height - size_filter_Y) / nb_iterations)

    fig = plt.figure(figsize=(25, 25))
    
    for ite in range(nb_iterations) :
        
        height, width = gray_image.shape
        
        if(ite == nb_iterations-1):
            gray_image = cv2.resize(gray_image, (size_filter_X, size_filter_Y)) # Last iteration
            size_filter_X_new = width - nbPixelToReduceX
            size_filter_Y_new = height - nbPixelToReduceY
            height, width = gray_image.shape
        elif(ite == 0):
            size_filter_X_new = size_filter_X - nbPixelToReduceX
            size_filter_Y_new = size_filter_Y - nbPixelToReduceY
        else :
            gray_image = cv2.resize(gray_image, (width - nbPixelToReduceX, height - nbPixelToReduceY))   
        
        gray_imag_rect_copy  = copy.deepcopy(gray_image)
            
        # we get all square that seems to be a visage.
        result, listImgPos = search_visage(gray_image, size_filter_X, size_filter_Y, model)
        
        size_filter_X_new = nbPixelToReduceX + size_filter_X_new
        size_filter_Y_new = nbPixelToReduceY + size_filter_Y_new
        
        
        # draw red rectangle.
        for x, y in result: 
            gray_imag_rect = draw_rectangle(gray_image, x, y, size_filter_X, size_filter_Y)
        
        
        ## Display positives images
        #for l in listImgPos :
        #    plt.figure()
        #    plt.imshow(l)
            
        ## Display initial image with reds rectangles. 
        if(result == []) :
            sub.imshow(gray_image)
        else :
            sub.imshow(gray_imag_rect)
        
        # Remove rectangle before next iteration.
        gray_image = gray_imag_rect_copy       
        
    return gray_imag_rect

In [ ]:
model = load_model_keras('my_model.h5')
result = pyramid_search('hulk.jpg', SIZE_FILTER_X, SIZE_FILTER_Y, NB_ITERATION, model)

In [ ]:
#plt.imshow(result, cmap='gray')